# Производительность модели ключ-значение на примере работы с Redis

<p> https://en.wikipedia.org/wiki/Redis
<p> https://github.com/andymccurdy/redis-py

```python
'css:key': {
    "reg_id": <string>,
    "reg_name": <string>,
    "year": <number>,
    "group_id": <number>,
    "group_name": <string>,
    "category": <string>,
    "gender": <string>,
    "value": <number>,
}
```

In [1]:
import redis

In [2]:
pool = redis.ConnectionPool(decode_responses=True)
r = redis.StrictRedis(connection_pool=pool)

In [3]:
r.hgetall('css:0')

{'reg_id': '1100',
 'reg_name': 'Российская Федерация',
 'year': '2016',
 'group_id': '14002',
 'group_name': 'по возрасту',
 'category': '14-15',
 'gender': 'Мужчины',
 'value': '13573'}

In [4]:
r.hvals('css:0')

['1100',
 'Российская Федерация',
 '2016',
 '14002',
 'по возрасту',
 '14-15',
 'Мужчины',
 '13573']

In [5]:
r.hkeys('css:0')

['reg_id',
 'reg_name',
 'year',
 'group_id',
 'group_name',
 'category',
 'gender',
 'value']

In [6]:
r.hmget('css:0', 'reg_id', 'reg_name')

['1100', 'Российская Федерация']

In [7]:
%timeit r.hmget('css:0', 'reg_id', 'reg_name')

398 µs ± 119 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
